In [2]:
import io
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression 

###########################################################################
#generate training/test data
########################################################################### 
bragging_file = io.open("Happy_clean.txt",'r',encoding="utf-8") 
bitching_file = io.open("FML_clean.txt",'r',encoding="utf-8") 


bragging_tweets = []
bitching_tweets = []

for _ in range(249000):
    bragging_tweets.append([bragging_file.readline().strip("\n"),0])
    bitching_tweets.append([bitching_file.readline().strip("\n"),1])
    
tweets = bragging_tweets+bitching_tweets
np.random.seed(42)
np.random.shuffle(tweets)


tweets_train = np.array(tweets[:348600])
tweets_test = np.array(tweets[348600:498000])

X_train = tweets_train[:,0]
Y_train = tweets_train[:,1]
X_test = tweets_test[:,0]
Y_test = tweets_test[:,1]

#########################################################################
#bag of words
#########################################################################

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


In [13]:

import keras.optimizers
from keras.models import Sequential 
from keras.layers import Dense, Activation 

model = Sequential() 
model.add( Dense(units=1, activation="sigmoid", input_dim= X_train_counts.shape[1], use_bias=False) )

In [14]:
sgd = keras.optimizers.SGD(lr=0.0001*400)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_log = model.fit(X_train_counts, Y_train, batch_size= X_train_counts.shape[0], epochs=300)

Epoch 1/300


In [1]:
def calc_classification_error(predictions, class_labels):
    n = predictions.size
    num_of_errors = 0.
    for idx in range(n):
        if (predictions[idx] >= 0.5 and class_labels[idx]==0) or (predictions[idx] < 0.5 and class_labels[idx]==1):
            num_of_errors += 1
    return num_of_errors/n

In [ ]:
predictions = model.predict(features)
print("Classification Error on Training Set: %.2f%%" %(calc_classification_error(predictions, class_labels) * 100))